In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

sys.path.insert(0, "..")

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from datetime import datetime, timedelta
from pathlib import Path

import hvplot.polars
import numpy as np
import polars as pl
from sqlalchemy import text

In [6]:
from tsdb_benchmarks.monetdb import MonetDB
from tsdb_benchmarks.monetdb.fetch import fetch_binary, fetch_pymonetdb
from tsdb_benchmarks.monetdb.insert import insert
from tsdb_benchmarks.monetdb.utils import drop_table
from tsdb_benchmarks.settings import setup_stdout_logging

setup_stdout_logging()
db = MonetDB()

In [ ]:
query = """

with hourly_stats as (
select
	date_trunc('hour', event_created) as "hour",
	trim(json.filter(event_payload,
	'$.terminal'), '"') as terminal,
	count(*) as event_count
from
	order_events
where
	event_created >= '2024-01-01'
	and event_created < '2024-02-01'
	and event_type in ('Created', 'Departed', 'Delivered')
group by
	"hour",
	terminal
)
select
	"hour",
	terminal,
	event_count,
	avg(event_count) over (
    partition by terminal
order by
	"hour"
    rows between 3 preceding and current row
  ) as moving_avg_events
from
	hourly_stats
where
	terminal in ('Berlin', 'Hamburg', 'Munich')
order by
	terminal,
	"hour"

"""

query = """

select
	date_trunc('day', event_created) as "day",
	count(*) as count
from
	order_events
where
	event_created >= '2024-04-01'
	and event_created < '2024-05-01'
	and event_type = 'Departed'
	and trim(json.filter(event_payload,
	'$.terminal'), '"') = 'Berlin'
group by
	"day"
order by
	count desc,
	"day";


"""


query = """

SELECT * FROM order_events
WHERE event_created >= '2024-03-01' and event_created < '2024-08-01'
AND processor LIKE '%ron%' ORDER BY event_created LIMIT 10;


"""

fetch_pymonetdb(query, db.connect())

order_id,counter,event_created,event_type,satisfaction,processor,backup_processor,event_payload
i32,i32,datetime[ms],str,f32,str,str,struct[0]
1455600,15,2024-03-01 00:05:44,"""Delivered""",28.99,"""ExjronD WfxUwSB""",null,{}
1641397,0,2024-03-01 00:18:48,"""Created""",8.94,"""Kjrondn MvrUSnt""",null,{}
1591709,8,2024-03-01 00:33:39,"""Arrived""",71.580002,"""PronSwm cTwNpOq""",null,{}
1588946,7,2024-03-01 00:57:57,"""Departed""",30.17,"""Sronshi olFRWJg""",null,{}
1439458,25,2024-03-01 01:14:51,"""Arrived""",14.73,"""PLtltQM pbdronv""",null,{}
1628768,3,2024-03-01 01:15:23,"""Approved""",73.010002,"""PronGCR EoAVVqN""",null,{}
1582794,9,2024-03-01 01:29:49,"""Approved""",92.050003,"""iDucbxp BtjGron""",null,{}
1579040,11,2024-03-01 02:09:06,"""Departed""",11.9,"""txEronJ hVQfpgD""",null,{}
1574658,12,2024-03-01 03:37:56,"""Approved""",64.980003,"""WjdHUUF ronSvBt""",null,{}


In [8]:
with Path("../tsdb_benchmarks/schemas/monetdb.sql").open() as f:
    sql = f.read()

con = db.connect()
con.execute(text(sql))

con.commit()

In [9]:
from tsdb_benchmarks.generate import RTA_BENCH_SCHEMAS

# for name in RTA_BENCH_SCHEMAS:
#     df = pl.read_parquet(f"../data/input/rtabench/{name}.parquet")
#     insert(df, name, db.connect(), create=False)
#     print(f"Inserted {name}")